# Collaborators Whereabouts
Author: Sam Eriksen

Description: Notebook to read in collaboration list from google sheets and plot

In [1]:
import numpy as np
import pandas as pd

Get author list information

In [70]:
spreadsheet_id = "1J-8ehKgEcpmssEZ_dGRIp5lRQCg6tDonYYG87Rv__4I"
sheet_id = "951549455"
sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}"

df = pd.read_csv(sheet_url)

cut_down_df = df.loc[4:].dropna(subset=["Unnamed: 2"])

final_info = cut_down_df[["Unnamed: 47", "Unnamed: 43", "# considered (x-check)", "PRIMARY PAGE", "Unnamed: 3"]]

final_info.columns = ["Title", "Initials", "LastName", "University", "isAuthor"]

In [71]:
final_info

,Title,Initials,LastName,University,isAuthor
6,Faculty,B. J.,Mount,Black Hills State University,TRUE
7,Undergrad,A.,Rodriguez,Black Hills State University,FALSE
9,Grad Stud,L.,Korley,Brandeis University,FALSE
10,Faculty,B.,Penning,Brandeis University,FALSE
11,Post Doc,J.J.,Wang,Brandeis University,FALSE
...,...,...,...,...,...
832,Engineer,W. T.,Emmet,Yale University,FALSE
833,Technician,T. W.,Hurteau,Yale University,FALSE
834,Grad Stud,N. A.,Larsen,Yale University,FALSE
835,Grad Stud,B. P.,Tennyson,Yale University,FALSE


Get university address information

In [111]:
spreadsheet_id = "1J-8ehKgEcpmssEZ_dGRIp5lRQCg6tDonYYG87Rv__4I"
sheet_id = "1098381277"
sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}"

df = pd.read_csv(sheet_url)

institutions = df[["SECONDARY PAGE", "Unnamed: 7"]]
institutions.columns = ["University", "Address"]
institutions = institutions.iloc[4:]
institutions = institutions.dropna(subset=['Address'])
institutions['Universities_FullName'] = institutions['University'] + ', ' + institutions['Address']

Plot on a map

In [81]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
def get_latitude(location):
    try:
        return location.latitude
    except AttributeError:
        return None

def get_longitude(location):
    try:
        return location.longitude
    except AttributeError:
        return None

In [84]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="university_geocoder")
# To avoid hitting the service rate limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [113]:
institutions['location'] = institutions['Universities_FullName'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Black Hills State University, School of Natural Sciences, 1200 University Street, Spearfish, SD 57799-0002, USA',), **{}).
Traceback (most recent call last):
  File "/home/sam/lz/sustainable-lz/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "/home/sam/lz/sustainable-lz/venv/lib/python3.10/site-packages/urllib3/connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/

In [105]:
institutions

,university,location
0,Black Hills State University,"(Black Hills State University, 1200, Universit..."
1,Brandeis University,"(Brandeis University, 415, South Street, West ..."
2,Brookhaven National Laboratory (BNL),"(BNL Police, Upton Road, Brookhaven National L..."
3,Brown University,"(Brown University, Eddy Street, Downtown, Prov..."
4,Center for Underground Physics (CUP),None
5,Fermi National Accelerator Laboratory (FNAL),"(Fermi National Accelerator Laboratory, Wilson..."
6,Imperial College London,"(Imperial College London, Exhibition Road, Kni..."
7,Laboratório de Instrumentação e Física Experim...,None
8,Lawrence Berkeley National Laboratory (LBNL),"(Lawrence Berkeley National Laboratory, Centen..."
9,Lawrence Livermore National Laboratory (LLNL),"(Lawrence Livermore National Laboratory, Sixth..."
